In [5]:
import json
import numpy as np
from sklearn.model_selection import StratifiedGroupKFold

# load json: modify the path to your own ‘train.json’ file
annotation = '/opt/ml/dataset/train.json'

with open(annotation) as f: data = json.load(f)

var = [(ann['image_id'], ann['category_id']) for ann in data['annotations']]
X = np.ones((len(data['annotations']),1))
y = np.array([v[1] for v in var])
groups = np.array([v[0] for v in var])

cv = StratifiedGroupKFold(n_splits=5, shuffle=True, random_state=411)

for train_idx, val_idx in cv.split(X, y, groups):
    print("TRAIN:", groups[train_idx])
    print(" ", y[train_idx])
    print(" TEST:", groups[val_idx])
    print(" ", y[val_idx])

TRAIN: [   0    1    1 ... 4881 4881 4881]
  [0 3 7 ... 7 1 7]
 TEST: [   6   13   13 ... 4882 4882 4882]
  [1 6 7 ... 0 1 1]
TRAIN: [   0    1    1 ... 4882 4882 4882]
  [0 3 7 ... 0 1 1]
 TEST: [   5    5    5 ... 4876 4876 4878]
  [7 0 0 ... 0 2 0]
TRAIN: [   0    3    3 ... 4882 4882 4882]
  [0 2 6 ... 0 1 1]
 TEST: [   1    1    1 ... 4877 4877 4880]
  [3 7 4 ... 7 7 0]
TRAIN: [   1    1    1 ... 4882 4882 4882]
  [3 7 4 ... 0 1 1]
 TEST: [   0    3    3 ... 4881 4881 4881]
  [0 2 6 ... 7 1 7]
TRAIN: [   0    1    1 ... 4882 4882 4882]
  [0 3 7 ... 0 1 1]
 TEST: [   4    4    4 ... 4868 4872 4872]
  [1 1 1 ... 2 4 6]


In [6]:
# check distribution
import pandas as pd
from collections import Counter

def get_distribution(y):
    y_distr = Counter(y)
    y_vals_sum = sum(y_distr.values())

    return [f'{y_distr[i]/y_vals_sum:.2%}' for i in range(np.max(y) +1)]

distrs = [get_distribution(y)]
index = ['training set']

for fold_ind, (train_idx, val_idx) in enumerate(cv.split(X,y, groups)):
    train_y, val_y = y[train_idx], y[val_idx]
    train_gr, val_gr = groups[train_idx], groups[val_idx]

    assert len(set(train_gr) & set(val_gr)) == 0 
    distrs.append(get_distribution(train_y))

    distrs.append(get_distribution(val_y))
    index.append(f'train - fold{fold_ind}')
    index.append(f'val - fold{fold_ind}')

categories = [d['name'] for d in data['categories']]
pd.DataFrame(distrs, index=index, columns = [categories[i] for i in range(np.max(y) + 1)])

,General trash,Paper,Paper pack,Metal,Glass,Plastic,Styrofoam,Plastic bag,Battery,Clothing
training set,17.14%,27.45%,3.88%,4.04%,4.24%,12.72%,5.46%,22.37%,0.69%,2.02%
train - fold0,16.96%,27.45%,3.79%,4.13%,4.48%,12.61%,5.51%,22.28%,0.77%,2.02%
val - fold0,17.85%,27.42%,4.23%,3.70%,3.26%,13.15%,5.25%,22.77%,0.35%,2.02%
train - fold1,17.14%,27.24%,4.01%,3.98%,4.28%,12.77%,5.38%,22.32%,0.67%,2.20%
val - fold1,17.12%,28.17%,3.41%,4.26%,4.12%,12.51%,5.72%,22.57%,0.73%,1.38%
train - fold2,17.31%,27.39%,3.83%,4.08%,4.13%,12.80%,5.14%,22.68%,0.69%,1.94%
val - fold2,16.42%,27.68%,4.05%,3.88%,4.70%,12.36%,6.76%,21.12%,0.69%,2.35%
train - fold3,17.30%,27.47%,3.87%,4.06%,4.22%,12.63%,5.49%,22.39%,0.63%,1.95%
val - fold3,16.50%,27.36%,3.88%,3.99%,4.33%,13.07%,5.33%,22.30%,0.92%,2.32%
train - fold4,16.97%,27.67%,3.88%,3.97%,4.10%,12.77%,5.76%,22.20%,0.68%,2.00%


In [19]:
next(cv.split(X,y, groups))[0]

array([    0,     1,     2, ..., 23136, 23137, 23138])

In [25]:
len(set(train_gr))

3909

In [50]:
import random
import os
import shutil

origin_dataset_dir = '/opt/ml/dataset'
new_dataset_dir = '/opt/ml/test_dir'
input_json_path = '/opt/ml/dataset/train.json' #train.json 파일 경로
val_ratio = 0.2


for fold_ind, (train_idx, val_idx) in enumerate(cv.split(X,y, groups)):
#json 파일 불러오기
    with open(input_json_path, 'r') as json_reader:
        dataset = json.load(json_reader)

    images = dataset['images'] # dict에서 (key:images)의 values 불러오기
    categories = dataset['categories']# dict에서 (key:catagories)의 values 불러오기
    annotations = dataset['annotations']# dict에서 (key:annotations)의 values 불러오기
    
    train_gr, val_gr = groups[train_idx], groups[val_idx]

    # image_ids = [x.get('id') for x in images] # get함수를 통해 dict에서 id값 추출
    # image_ids.sort() # 정렬
    # random.shuffle(image_ids) # 인덱스 섞기

    # num_val = int(len(image_ids) * val_ratio)
    # num_train = len(image_ids) - num_val

    image_ids_val, image_ids_train = set(val_gr), set(train_gr)

    num_train = len(image_ids_train)
    num_val = len(image_ids_val)

    #Image_id를 기준으로 train/val 나누기
    train_images = [x for x in images if x.get('id') in image_ids_train]
    val_images = [x for x in images if x.get('id') in image_ids_val]
    train_annotations = [x for x in annotations if x.get('image_id') in image_ids_train]
    val_annotations = [x for x in annotations if x.get('image_id') in image_ids_val]

    #file_name 수정
    for info in val_images:
        name = info['file_name'].split('/')[1]
        info['file_name'] = os.path.join('val',name)
        
    #나눈 정보를 가지고 새로운 dict 생성
    train_data = {
        'images': train_images,
        'annotations': train_annotations,
        'categories': categories,
    }

    val_data = {
        'images': val_images,
        'annotations': val_annotations,
        'categories': categories,
    }


    # 새롭게 만든 dict로 train/val json 파일 생성
    os.makedirs(new_dataset_dir+f'/{fold_ind}', exist_ok=True)

    new_train_json = os.path.join(new_dataset_dir, f'{fold_ind}','train.json')
    new_val_json = os.path.join(new_dataset_dir,f'{fold_ind}', 'val.json')
    copy_test_json = os.path.join(new_dataset_dir, f'{fold_ind}','test.json')

    #train.json 새롭게 생성
    with open(new_train_json, 'w') as train_writer:
        json.dump(train_data, train_writer)

    #val.json 새롭게 생성
    with open(new_val_json, 'w') as val_writer:
        json.dump(val_data, val_writer)

    # train/val 이미지 파일 분리 복사
    os.makedirs(os.path.join(new_dataset_dir, f'{fold_ind}','train'), exist_ok=True)
    os.makedirs(os.path.join(new_dataset_dir, f'{fold_ind}','val'), exist_ok=True)

    # train 해당 파일 복사
    for train_img_info in train_images:
        from_copy_train_img = os.path.join(origin_dataset_dir, train_img_info['file_name'])
        to_copy_train_img = os.path.join(new_dataset_dir, f'{fold_ind}',train_img_info['file_name'])
        shutil.copyfile(from_copy_train_img, to_copy_train_img)
        
    # val 해당 파일 복사
    for val_img_info in val_images:
        origin_id = os.path.join('train', val_img_info['file_name'].split('/')[1])
        from_copy_val_img = os.path.join(origin_dataset_dir, origin_id)
        to_copy_val_img = os.path.join(new_dataset_dir,f'{fold_ind}', val_img_info['file_name'])
        shutil.copyfile(from_copy_val_img, to_copy_val_img)
    
    #기존 파일에서 test json파일 복사
    shutil.copyfile(os.path.join(origin_dataset_dir, 'test.json'), copy_test_json)

    # test 이미지 폴더 전체 복사
    shutil.copytree(os.path.join(origin_dataset_dir, 'test'), os.path.join(new_dataset_dir,f'{fold_ind}', 'test'))


    print(f'train 이미지 파일 개수({int((1-val_ratio)*100)}%):{num_train}')
    print('new_dataset_train 파일 개수:{}'.format(len(os.listdir(os.path.join(new_dataset_dir,f'{fold_ind}','train')))))
    print(f'val 이미지 파일 개수({int(val_ratio*100)}%):{num_val}')
    print('new_dataset_val 파일 개수:{}'.format(len(os.listdir(os.path.join(new_dataset_dir,f'{fold_ind}', 'val')))))



train 이미지 파일 개수(80%):3914
new_dataset_train 파일 개수:3914
val 이미지 파일 개수(20%):969
new_dataset_val 파일 개수:969
train 이미지 파일 개수(80%):3906
new_dataset_train 파일 개수:3906
val 이미지 파일 개수(20%):977
new_dataset_val 파일 개수:977
train 이미지 파일 개수(80%):3901
new_dataset_train 파일 개수:3901
val 이미지 파일 개수(20%):982
new_dataset_val 파일 개수:982
train 이미지 파일 개수(80%):3902
new_dataset_train 파일 개수:3902
val 이미지 파일 개수(20%):981
new_dataset_val 파일 개수:981
train 이미지 파일 개수(80%):3909
new_dataset_train 파일 개수:3909
val 이미지 파일 개수(20%):974
new_dataset_val 파일 개수:974
